In [18]:
#  Install required libraries for API data collection

!pip install requests pandas tqdm


In [19]:
#Store OpenAQ API Key securely in Colab

import os

# Paste your OpenAQ API key between the quotes
os.environ["OPENAQ_API_KEY"] = "b48a0383e76fe19f1fb4c05963c9dd19717da1c126958abe297ff7c8d6e4006a"

# Simple verification (DO NOT print the key)
if "OPENAQ_API_KEY" in os.environ:
    print("OpenAQ API key loaded successfully.")
else:
    print("API key not found. Check the setup.")


OpenAQ API key loaded successfully.


In [20]:
# Define cities, pollutants, and base API configuration

# Major cities of India (expandable later if needed)
INDIAN_CITIES = [
    "Delhi",
    "Mumbai",
    "Bengaluru",
    "Chennai",
    "Kolkata",
    "Hyderabad",
    "Pune",
    "Ahmedabad",
    "Jaipur",
    "Lucknow"
]

# Pollutants required by the project
POLLUTANTS = ["pm25", "pm10", "no2", "so2", "co", "o3"]

# OpenAQ v3 base configuration
BASE_URL = "https://api.openaq.org/v3"
COUNTRY_CODE = "IN"

print("Cities configured:", len(INDIAN_CITIES))
print("Pollutants configured:", POLLUTANTS)
print("Base URL:", BASE_URL)


Cities configured: 10
Pollutants configured: ['pm25', 'pm10', 'no2', 'so2', 'co', 'o3']
Base URL: https://api.openaq.org/v3


In [24]:
# STEP 5A (FINAL): Fetch Indian locations using geographic bounding box

import requests
import os

API_KEY = os.environ.get("OPENAQ_API_KEY")

headers = {
    "X-API-Key": API_KEY
}

locations_url = f"{BASE_URL}/locations"

# India bounding box: [min_lon, min_lat, max_lon, max_lat]
params = {
    "bbox": "68,6,97,37",
    "limit": 10
}

response = requests.get(locations_url, headers=headers, params=params)

print("HTTP Status Code:", response.status_code)

if response.status_code == 200:
    data = response.json()
    print("Locations fetched:", len(data["results"]))
    print("Sample VERIFIED Indian location:")
    print(data["results"][0])
else:
    print("Failed to fetch locations")
    print(response.text)


HTTP Status Code: 200
Locations fetched: 10
Sample VERIFIED Indian location:
{'id': 12, 'name': 'SPARTAN - IIT Kanpur', 'locality': None, 'timezone': 'Asia/Kolkata', 'country': {'id': 9, 'code': 'IN', 'name': 'India'}, 'owner': {'id': 4, 'name': 'Unknown Governmental Organization'}, 'provider': {'id': 226, 'name': 'Spartan'}, 'isMobile': False, 'isMonitor': True, 'instruments': [{'id': 2, 'name': 'Government Monitor'}], 'sensors': [{'id': 23, 'name': 'pm25 µg/m³', 'parameter': {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': 'PM2.5'}}], 'coordinates': {'latitude': 26.519, 'longitude': 80.233}, 'licenses': None, 'bounds': [80.233, 26.519, 80.233, 26.519], 'distance': None, 'datetimeFirst': None, 'datetimeLast': None}


In [29]:
# STEP 5B (FINAL): Discover sensors via locations and fetch measurements

import requests
import os

API_KEY = os.environ.get("OPENAQ_API_KEY")

headers = {
    "X-API-Key": API_KEY
}

# 1️⃣ Fetch Indian locations
locations_url = f"{BASE_URL}/locations"

params = {
    "bbox": "68,6,97,37",   # India bounding box
    "limit": 20
}

response = requests.get(locations_url, headers=headers, params=params)

print("Locations status:", response.status_code)

if response.status_code != 200:
    print("Failed to fetch locations")
    raise SystemExit

locations = response.json()["results"]

print("Locations retrieved:", len(locations))

# 2️⃣ Find first sensor inside locations
sensor_id = None

for loc in locations:
    sensors = loc.get("sensors", [])
    if sensors:
        sensor_id = sensors[0]["id"]
        print("Using sensor ID:", sensor_id)
        break

if sensor_id is None:
    print("No sensors found in locations")
    raise SystemExit

# 3️⃣ Fetch measurements from that sensor
measurements_url = f"{BASE_URL}/sensors/{sensor_id}/measurements"

params = {
    "limit": 10,
    "sort": "desc"
}

response = requests.get(measurements_url, headers=headers, params=params)

print("Measurements status:", response.status_code)

if response.status_code == 200:
    results = response.json()["results"]
    print("Measurements fetched:", len(results))

    if len(results) > 0:
        print("Sample measurement:")
        print(results[0])
    else:
        print("Sensor exists but has no recent data")
else:
    print("Failed to fetch measurements")
    print(response.text)


Locations status: 200
Locations retrieved: 20
Using sensor ID: 23
Measurements status: 200
Measurements fetched: 0
Sensor exists but has no recent data


In [30]:
# STEP 6: Define final dataset schema for INDIA_openaq_pollution.csv

FINAL_COLUMNS = [
    "city",
    "location_id",
    "location_name",
    "sensor_id",
    "pollutant",
    "value",
    "unit",
    "latitude",
    "longitude",
    "timestamp_utc",
    "country",
    "source"
]

print("Final dataset schema defined.")
print("Number of columns:", len(FINAL_COLUMNS))
print(FINAL_COLUMNS)


Final dataset schema defined.
Number of columns: 12
['city', 'location_id', 'location_name', 'sensor_id', 'pollutant', 'value', 'unit', 'latitude', 'longitude', 'timestamp_utc', 'country', 'source']


In [31]:
# STEP 7: Controlled data collection loop (India only, live API)

import requests
import os
from tqdm import tqdm

API_KEY = os.environ.get("OPENAQ_API_KEY")

headers = {
    "X-API-Key": API_KEY
}

# Storage for collected rows (in-memory only)
collected_rows = []

# 1️⃣ Fetch Indian locations (limited for safety)
locations_url = f"{BASE_URL}/locations"

location_params = {
    "bbox": "68,6,97,37",   # India bounding box
    "limit": 50            # controlled limit (can be increased later)
}

loc_response = requests.get(locations_url, headers=headers, params=location_params)
loc_response.raise_for_status()

locations = loc_response.json()["results"]

print("Total Indian locations fetched:", len(locations))

# 2️⃣ Loop through locations → sensors → measurements
for loc in tqdm(locations, desc="Processing locations"):
    location_id = loc.get("id")
    location_name = loc.get("name")
    coords = loc.get("coordinates", {})
    latitude = coords.get("latitude")
    longitude = coords.get("longitude")

    sensors = loc.get("sensors", [])

    for sensor in sensors:
        sensor_id = sensor.get("id")
        pollutant = sensor.get("parameter", {}).get("name")

        # Only required pollutants
        if pollutant not in POLLUTANTS:
            continue

        measurements_url = f"{BASE_URL}/sensors/{sensor_id}/measurements"

        meas_params = {
            "limit": 5,
            "sort": "desc"
        }

        meas_response = requests.get(measurements_url, headers=headers, params=meas_params)

        if meas_response.status_code != 200:
            continue

        results = meas_response.json().get("results", [])

        # Skip sensors with no data
        if not results:
            continue

        for r in results:
            collected_rows.append({
                "city": None,  # city mapping comes later if needed
                "location_id": location_id,
                "location_name": location_name,
                "sensor_id": sensor_id,
                "pollutant": r.get("parameter"),
                "value": r.get("value"),
                "unit": r.get("unit"),
                "latitude": latitude,
                "longitude": longitude,
                "timestamp_utc": r.get("date", {}).get("utc"),
                "country": "IN",
                "source": "OpenAQ v3"
            })

print("Total records collected (in memory):", len(collected_rows))


Total Indian locations fetched: 50


Processing locations: 100%|██████████| 50/50 [01:37<00:00,  1.95s/it]

Total records collected (in memory): 521


In [32]:
# STEP 8: Convert collected data to DataFrame and save CSV

import pandas as pd

# Create DataFrame using predefined schema
df = pd.DataFrame(collected_rows, columns=FINAL_COLUMNS)

# Save to CSV
csv_filename = "INDIA_openaq_pollution.csv"
df.to_csv(csv_filename, index=False)

print("CSV file saved:", csv_filename)
print("Total rows:", df.shape[0])
print("Total columns:", df.shape[1])

# Preview first 5 rows
df.head()


CSV file saved: INDIA_openaq_pollution.csv
Total rows: 521
Total columns: 12


,city,location_id,location_name,sensor_id,pollutant,value,unit,latitude,longitude,timestamp_utc,country,source
0,None,13,"Delhi Technological University, Delhi - CPCB",13866,"{'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...",11.3,None,28.744,77.12,None,IN,OpenAQ v3
1,None,13,"Delhi Technological University, Delhi - CPCB",13866,"{'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...",8.6,None,28.744,77.12,None,IN,OpenAQ v3
2,None,13,"Delhi Technological University, Delhi - CPCB",13866,"{'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...",8.6,None,28.744,77.12,None,IN,OpenAQ v3
3,None,13,"Delhi Technological University, Delhi - CPCB",13866,"{'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...",9.3,None,28.744,77.12,None,IN,OpenAQ v3
4,None,13,"Delhi Technological University, Delhi - CPCB",13866,"{'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...",10.6,None,28.744,77.12,None,IN,OpenAQ v3


In [33]:
#  Load dataset and inspect structure

import pandas as pd

# Load the CSV generated in Milestone-1
df = pd.read_csv("INDIA_openaq_pollution.csv")

# Basic inspection
print("Dataset shape (rows, columns):", df.shape)
print("\nColumn names:")
print(df.columns.tolist())

print("\nData types:")
print(df.dtypes)

print("\nFirst 5 rows:")
df.head()


Dataset shape (rows, columns): (521, 12)

Column names:
['city', 'location_id', 'location_name', 'sensor_id', 'pollutant', 'value', 'unit', 'latitude', 'longitude', 'timestamp_utc', 'country', 'source']

Data types:
city             float64
location_id        int64
location_name     object
sensor_id          int64
pollutant         object
value            float64
unit             float64
latitude         float64
longitude        float64
timestamp_utc    float64
country           object
source            object
dtype: object

First 5 rows:


,city,location_id,location_name,sensor_id,pollutant,value,unit,latitude,longitude,timestamp_utc,country,source
0,NaN,13,"Delhi Technological University, Delhi - CPCB",13866,"{'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...",11.3,NaN,28.744,77.12,NaN,IN,OpenAQ v3
1,NaN,13,"Delhi Technological University, Delhi - CPCB",13866,"{'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...",8.6,NaN,28.744,77.12,NaN,IN,OpenAQ v3
2,NaN,13,"Delhi Technological University, Delhi - CPCB",13866,"{'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...",8.6,NaN,28.744,77.12,NaN,IN,OpenAQ v3
3,NaN,13,"Delhi Technological University, Delhi - CPCB",13866,"{'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...",9.3,NaN,28.744,77.12,NaN,IN,OpenAQ v3
4,NaN,13,"Delhi Technological University, Delhi - CPCB",13866,"{'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...",10.6,NaN,28.744,77.12,NaN,IN,OpenAQ v3


In [34]:
#  Parse and flatten the pollutant column

import ast

# Safely extract pollutant name and unit from the dict-like string
def parse_pollutant(p):
    try:
        p_dict = ast.literal_eval(p)
        return p_dict.get("name"), p_dict.get("units")
    except Exception:
        return None, None

df[["pollutant_name", "pollutant_unit"]] = df["pollutant"].apply(
    lambda x: pd.Series(parse_pollutant(x))
)

# Verify results
print("New columns added:")
print(df[["pollutant", "pollutant_name", "pollutant_unit"]].head())


New columns added:
                                           pollutant pollutant_name  \
0  {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...            no2   
1  {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...            no2   
2  {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...            no2   
3  {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...            no2   
4  {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'di...            no2   

  pollutant_unit  
0          µg/m³  
1          µg/m³  
2          µg/m³  
3          µg/m³  
4          µg/m³  


In [35]:
#  Fix data types (NO imputation yet)

# 1️⃣ Fix timestamp: convert to datetime (UTC)
df["timestamp_utc"] = pd.to_datetime(df["timestamp_utc"], errors="coerce", utc=True)

# 2️⃣ Fix unit: replace raw 'unit' column using parsed pollutant_unit
df["unit"] = df["pollutant_unit"]

# 3️⃣ Fix city column type (keep NaN, just correct dtype)
df["city"] = df["city"].astype("object")

# Verify data types after correction
print("Updated data types:")
print(df.dtypes)

print("\nSample rows after type fix:")
df[["pollutant_name", "value", "unit", "timestamp_utc", "city"]].head()


Updated data types:
city                           object
location_id                     int64
location_name                  object
sensor_id                       int64
pollutant                      object
value                         float64
unit                           object
latitude                      float64
longitude                     float64
timestamp_utc     datetime64[ns, UTC]
country                        object
source                         object
pollutant_name                 object
pollutant_unit                 object
dtype: object

Sample rows after type fix:


,pollutant_name,value,unit,timestamp_utc,city
0,no2,11.3,µg/m³,NaT,NaN
1,no2,8.6,µg/m³,NaT,NaN
2,no2,8.6,µg/m³,NaT,NaN
3,no2,9.3,µg/m³,NaT,NaN
4,no2,10.6,µg/m³,NaT,NaN


In [36]:
#  Handle missing values safely

# Count missing values before
print("Missing values BEFORE:")
print(df.isnull().sum())

# Drop rows where pollution value is missing
initial_rows = df.shape[0]
df = df.dropna(subset=["value"])
final_rows = df.shape[0]

print("\nRows before dropping missing values:", initial_rows)
print("Rows after dropping missing values:", final_rows)
print("Rows removed:", initial_rows - final_rows)

# Verify remaining missing values
print("\nMissing values AFTER:")
print(df.isnull().sum())


Missing values BEFORE:
city              521
location_id         0
location_name       0
sensor_id           0
pollutant           0
value               0
unit                0
latitude            0
longitude           0
timestamp_utc     521
country             0
source              0
pollutant_name      0
pollutant_unit      0
dtype: int64

Rows before dropping missing values: 521
Rows after dropping missing values: 521
Rows removed: 0

Missing values AFTER:
city              521
location_id         0
location_name       0
sensor_id           0
pollutant           0
value               0
unit                0
latitude            0
longitude           0
timestamp_utc     521
country             0
source              0
pollutant_name      0
pollutant_unit      0
dtype: int64


In [37]:
#  Remove duplicates and perform sanity checks

# 1️⃣ Remove exact duplicate rows
rows_before = df.shape[0]
df = df.drop_duplicates()
rows_after = df.shape[0]

print("Rows before duplicate removal:", rows_before)
print("Rows after duplicate removal:", rows_after)
print("Duplicates removed:", rows_before - rows_after)

# 2️⃣ Sanity check: value ranges by pollutant
print("\nSanity check (min / max by pollutant):")

sanity = (
    df.groupby("pollutant_name")["value"]
    .agg(["min", "max", "count"])
    .reset_index()
)

print(sanity)


Rows before duplicate removal: 521
Rows after duplicate removal: 440
Duplicates removed: 81

Sanity check (min / max by pollutant):
  pollutant_name      min      max  count
0             co    0.202  9600.00     76
1            no2    5.400   166.20     93
2             o3    1.300   181.33     84
3           pm10    5.800   582.00     37
4           pm25 -999.000   446.73     71
5            so2   -4.670    77.40     79


In [38]:
# Temporal feature extraction (safe)

# Create temporal features only where timestamp exists
df["hour"] = df["timestamp_utc"].dt.hour
df["day"] = df["timestamp_utc"].dt.day
df["month"] = df["timestamp_utc"].dt.month
df["day_of_week"] = df["timestamp_utc"].dt.dayofweek

# Verify temporal features
print("Temporal feature sample:")
df[["timestamp_utc", "hour", "day", "month", "day_of_week"]].head()

print("\nMissing values in temporal features:")
print(df[["hour", "day", "month", "day_of_week"]].isnull().sum())


Temporal feature sample:

Missing values in temporal features:
hour           440
day            440
month          440
day_of_week    440
dtype: int64


In [39]:
# Geographic sanity checks

# Check latitude and longitude ranges
lat_outliers = df[
    (df["latitude"] < 6) | (df["latitude"] > 37)
]

lon_outliers = df[
    (df["longitude"] < 68) | (df["longitude"] > 97)
]

print("Latitude out-of-range points:", lat_outliers.shape[0])
print("Longitude out-of-range points:", lon_outliers.shape[0])

# Summary statistics for coordinates
print("\nLatitude summary:")
print(df["latitude"].describe())

print("\nLongitude summary:")
print(df["longitude"].describe())


Latitude out-of-range points: 0
Longitude out-of-range points: 0

Latitude summary:
count    440.000000
mean      26.736190
std        3.992542
min       12.992514
25%       26.833997
50%       28.563262
75%       28.646835
max       30.705778
Name: latitude, dtype: float64

Longitude summary:
count    440.000000
mean      79.285607
std        3.548160
min       75.773878
25%       77.131023
50%       77.316032
75%       80.891736
max       90.423698
Name: longitude, dtype: float64


In [40]:
#  Prepare final cleaned dataset and save

# Select final columns for Milestone-2 output
final_columns_m2 = [
    "location_id",
    "location_name",
    "sensor_id",
    "pollutant_name",
    "value",
    "unit",
    "latitude",
    "longitude",
    "timestamp_utc",
    "hour",
    "day",
    "month",
    "day_of_week",
    "country",
    "source"
]

df_m2 = df[final_columns_m2].copy()

# Save Milestone-2 output
m2_filename = "INDIA_openaq_pollution_cleaned_M2.csv"
df_m2.to_csv(m2_filename, index=False)

print("Milestone-2 CSV saved:", m2_filename)
print("Final shape:", df_m2.shape)

# Preview final dataset
df_m2.head()


Milestone-2 CSV saved: INDIA_openaq_pollution_cleaned_M2.csv
Final shape: (440, 15)


,location_id,location_name,sensor_id,pollutant_name,value,unit,latitude,longitude,timestamp_utc,hour,day,month,day_of_week,country,source
0,13,"Delhi Technological University, Delhi - CPCB",13866,no2,11.3,µg/m³,28.744,77.12,NaT,NaN,NaN,NaN,NaN,IN,OpenAQ v3
1,13,"Delhi Technological University, Delhi - CPCB",13866,no2,8.6,µg/m³,28.744,77.12,NaT,NaN,NaN,NaN,NaN,IN,OpenAQ v3
3,13,"Delhi Technological University, Delhi - CPCB",13866,no2,9.3,µg/m³,28.744,77.12,NaT,NaN,NaN,NaN,NaN,IN,OpenAQ v3
4,13,"Delhi Technological University, Delhi - CPCB",13866,no2,10.6,µg/m³,28.744,77.12,NaT,NaN,NaN,NaN,NaN,IN,OpenAQ v3
5,13,"Delhi Technological University, Delhi - CPCB",13864,pm25,300.0,µg/m³,28.744,77.12,NaT,NaN,NaN,NaN,NaN,IN,OpenAQ v3


In [41]:
# Load cleaned dataset and define labeling objective

import pandas as pd

# Load Milestone-2 output
df = pd.read_csv("INDIA_openaq_pollution_cleaned_M2.csv")

print("Dataset loaded successfully.")
print("Shape:", df.shape)

print("\nColumns:")
print(df.columns.tolist())

# Define pollution source labels (fixed)
SOURCE_LABELS = [
    "Vehicular",
    "Industrial",
    "Agricultural",
    "Burning",
    "Natural",
    "Unknown"
]

print("\nDefined pollution source classes:")
print(SOURCE_LABELS)

# Preview data
df.head()


Dataset loaded successfully.
Shape: (440, 15)

Columns:
['location_id', 'location_name', 'sensor_id', 'pollutant_name', 'value', 'unit', 'latitude', 'longitude', 'timestamp_utc', 'hour', 'day', 'month', 'day_of_week', 'country', 'source']

Defined pollution source classes:
['Vehicular', 'Industrial', 'Agricultural', 'Burning', 'Natural', 'Unknown']


,location_id,location_name,sensor_id,pollutant_name,value,unit,latitude,longitude,timestamp_utc,hour,day,month,day_of_week,country,source
0,13,"Delhi Technological University, Delhi - CPCB",13866,no2,11.3,µg/m³,28.744,77.12,NaN,NaN,NaN,NaN,NaN,IN,OpenAQ v3
1,13,"Delhi Technological University, Delhi - CPCB",13866,no2,8.6,µg/m³,28.744,77.12,NaN,NaN,NaN,NaN,NaN,IN,OpenAQ v3
2,13,"Delhi Technological University, Delhi - CPCB",13866,no2,9.3,µg/m³,28.744,77.12,NaN,NaN,NaN,NaN,NaN,IN,OpenAQ v3
3,13,"Delhi Technological University, Delhi - CPCB",13866,no2,10.6,µg/m³,28.744,77.12,NaN,NaN,NaN,NaN,NaN,IN,OpenAQ v3
4,13,"Delhi Technological University, Delhi - CPCB",13864,pm25,300.0,µg/m³,28.744,77.12,NaN,NaN,NaN,NaN,NaN,IN,OpenAQ v3


In [42]:
#  Rule-based pollution source labeling

def assign_source_label(row):
    pollutant = row["pollutant_name"]
    value = row["value"]

    # Safety check
    if pd.isna(pollutant) or pd.isna(value):
        return "Unknown"

    # Vehicular emissions
    if pollutant in ["no2", "co"] and value >= 40:
        return "Vehicular"

    # Industrial emissions
    if pollutant == "so2" and value >= 20:
        return "Industrial"

    # Burning (extreme PM2.5)
    if pollutant == "pm25" and value >= 250:
        return "Burning"

    # Agricultural (high PM)
    if pollutant in ["pm25", "pm10"] and value >= 100:
        return "Agricultural"

    # Natural background
    if pollutant in ["pm25", "pm10"] and value < 100:
        return "Natural"

    # Fallback
    return "Unknown"


# Apply labeling
df["source_label"] = df.apply(assign_source_label, axis=1)

# Verify results
print("Source label distribution:")
print(df["source_label"].value_counts())

df[["pollutant_name", "value", "source_label"]].head(10)


Source label distribution:
source_label
Unknown         210
Vehicular        99
Agricultural     53
Natural          48
Industrial       23
Burning           7
Name: count, dtype: int64


,pollutant_name,value,source_label
0,no2,11.3,Unknown
1,no2,8.6,Unknown
2,no2,9.3,Unknown
3,no2,10.6,Unknown
4,pm25,300.0,Burning
5,pm25,93.0,Natural
6,co,5900.0,Vehicular
7,co,7800.0,Vehicular
8,co,8700.0,Vehicular
9,co,8400.0,Vehicular


In [43]:
#  Save final labeled dataset

# Select final columns for Milestone-3
final_columns_m3 = [
    "location_id",
    "location_name",
    "sensor_id",
    "pollutant_name",
    "value",
    "unit",
    "latitude",
    "longitude",
    "timestamp_utc",
    "hour",
    "day",
    "month",
    "day_of_week",
    "source_label",
    "country",
    "source"
]

df_m3 = df[final_columns_m3].copy()

# Save CSV
m3_filename = "INDIA_openaq_pollution_labeled_M3.csv"
df_m3.to_csv(m3_filename, index=False)

print("Milestone-3 CSV saved:", m3_filename)
print("Final shape:", df_m3.shape)

# Preview
df_m3.head()


Milestone-3 CSV saved: INDIA_openaq_pollution_labeled_M3.csv
Final shape: (440, 16)


,location_id,location_name,sensor_id,pollutant_name,value,unit,latitude,longitude,timestamp_utc,hour,day,month,day_of_week,source_label,country,source
0,13,"Delhi Technological University, Delhi - CPCB",13866,no2,11.3,µg/m³,28.744,77.12,NaN,NaN,NaN,NaN,NaN,Unknown,IN,OpenAQ v3
1,13,"Delhi Technological University, Delhi - CPCB",13866,no2,8.6,µg/m³,28.744,77.12,NaN,NaN,NaN,NaN,NaN,Unknown,IN,OpenAQ v3
2,13,"Delhi Technological University, Delhi - CPCB",13866,no2,9.3,µg/m³,28.744,77.12,NaN,NaN,NaN,NaN,NaN,Unknown,IN,OpenAQ v3
3,13,"Delhi Technological University, Delhi - CPCB",13866,no2,10.6,µg/m³,28.744,77.12,NaN,NaN,NaN,NaN,NaN,Unknown,IN,OpenAQ v3
4,13,"Delhi Technological University, Delhi - CPCB",13864,pm25,300.0,µg/m³,28.744,77.12,NaN,NaN,NaN,NaN,NaN,Burning,IN,OpenAQ v3
